In [49]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [ ]:

# load and prepare data
df = pd.read_csv('shopping_behavior_updated.csv')
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
seasonal_data = df.groupby(['Season', 'Category']).agg({
    'Purchase Amount (USD)': ['sum', 'mean', 'count']
}).reset_index()

In [ ]:
#store and put data
volume_data = []
average_data = []
count_data = []

for category in df['Category'].unique():
    cat_data = seasonal_data[seasonal_data['Category'] == category]
    volume_data.append(cat_data['Purchase Amount (USD)']['sum'].values)
    average_data.append(cat_data['Purchase Amount (USD)']['mean'].values)
    count_data.append(cat_data['Purchase Amount (USD)']['count'].values)

In [ ]:

# create fig
fig = go.Figure()


for category in df['Category'].unique():
    category_data = seasonal_data[seasonal_data['Category'] == category]

    # seasons to angles
    angles = category_data['Season'].map({
        'Spring': 90,
        'Summer': 0,
        'Fall': 270,
        'Winter': 180
    })

    #scatter plot
    fig.add_trace(go.Scatterpolar(
        r=category_data['Purchase Amount (USD)']['sum'],
        theta=angles,
        mode='markers',
        name=category,
        marker=dict(
            size=25,
            opacity=0.8,
            line=dict(color='white', width=1)
        ),
        customdata=np.stack((
            category_data['Purchase Amount (USD)']['sum'],
            category_data['Purchase Amount (USD)']['mean'],
            category_data['Purchase Amount (USD)']['count']
        ), axis=-1),
        hovertemplate=(
                "<b>%{text}</b><br>" +
                "Season: %{theta}<br>" +
                "Total Sales: $%{customdata[0]:,.2f}<br>" +
                "Average Purchase: $%{customdata[1]:,.2f}<br>" +
                "Number of Sales: %{customdata[2]}<br>" +
                "<extra></extra>"
        ),
        text=[f"{category} - {season}" for season in category_data['Season']]
    ))

# set layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            tickformat='$,.0f',
            title='Sales Volume',
            titlefont=dict(size=12),
            gridcolor='rgba(255, 255, 255, 0.2)'
        ),
        angularaxis=dict(
            ticktext=seasons,
            tickvals=[90, 0, 270, 180],
            direction='clockwise',
            gridcolor='rgba(255, 255, 255, 0.2)'
        ),
        bgcolor='rgba(0,0,0,0)'
    ),
    showlegend=True,
    legend=dict(
        x=1.1,
        y=0.5,
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(255,255,255,0.3)',
        borderwidth=1
    ),
    # Add buttons for different metrics
    updatemenus=[dict(
        type='buttons',
        showactive=True,
        buttons=[
            dict(
                label='Sales Volume',
                method='update',
                args=[{'r': volume_data},
                      {'polar.radialaxis.title': 'Sales Volume'}]
            ),
            dict(
                label='Average Purchase',
                method='update',
                args=[{'r': average_data},
                      {'polar.radialaxis.title': 'Average Purchase Amount'}]
            ),
            dict(
                label='Transaction Count',
                method='update',
                args=[{'r': count_data},
                      {'polar.radialaxis.title': 'Number of Transactions'}]
            )
        ],
        direction='down',
        x=0.1,
        y=1.1,
    )],
    width=900,
    height=700,
    title=dict(
        text='Seasonal Sales Analysis by Category',
        x=0.5,
        y=0.95
    ),
    margin=dict(l=80, r=150, t=100, b=80)
)

In [50]:

fig.show()